## 2.数据预处理
为了能⽤深度学习来解决现实世界的问题，我们经常从预处理原始数据开始，⽽不是从那些准备好的张量格式数据开始，常用数据处理工具有numpy,pandas等

### 2.2.1 读取数据集
举⼀个例⼦，我们⾸先创建⼀个⼈⼯数据集，并存储在CSV（逗号分隔值）⽂件 ../data/house_tiny.csv中。以其他格式存储的数据也可以通过类似的⽅式进⾏处理。下⾯我们将数据集按⾏写⼊CSV⽂件中。

In [1]:
import os

os.makedirs(os.path.join('..','data'),exist_ok=True)
data_file = os.path.join('..','data','house_tiny.csv')
with open(data_file,'w') as f:
    f.write('NumRooms, Alley, Prices\n') # 列名
    f.write('NA,Pave,127500\n') # 没行表示一个样本
    f.write('1,Pave,124500\n') 
    f.write('3,NA,123200\n') 
    f.write('NA,NA,123200\n') 

要从创建的CSV⽂件中加载原始数据集，我们导⼊pandas包并调⽤read_csv函数。该数据集有四⾏三列。其
中每⾏描述了房间数量（“NumRooms”）、巷⼦类型（“Alley”）和房屋价格（“Price”）。

In [2]:
import pandas as pd

print('data dir',data_file)
data = pd.read_csv(data_file)
print(data)

data dir ..\data\house_tiny.csv
   NumRooms  Alley   Prices
0       NaN   Pave   127500
1       1.0   Pave   124500
2       3.0    NaN   123200
3       NaN    NaN   123200


 ### 2.2.2 处理缺失值
 注意，“NaN”项代表缺失值。 为了处理缺失的数据，典型的方法包括插值法和删除法， 其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。 在这里，我们将考虑插值法。

通过位置索引iloc，我们将data分成inputs和outputs， 其中前者为data的前两列，而后者为data的最后一列。 对于inputs中缺少的数值，我们用同一列的均值替换“NaN”项。

In [3]:
inputs,outputs = data.iloc[:,0:2],data.iloc[:,2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms  Alley
0       2.0   Pave
1       1.0   Pave
2       3.0    NaN
3       2.0    NaN


C:\Users\yangq\AppData\Local\Temp\ipykernel_1420\3782217204.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  inputs = inputs.fillna(inputs.mean())


对于inputs中的类别值或离散值，我们将“NaN”视为一个类别。 由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”， pandas可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。 巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。 缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。

In [4]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms   Alley_Pave   Alley_nan
0       2.0            1           0
1       1.0            1           0
2       3.0            0           1
3       2.0            0           1


### 2.2.3 转换为张量格式
现在inputs和outputs中的所有条目都是数值类型，它们可以转换为张量格式。 当数据采用张量格式后，可以通过在 2.1节中引入的那些张量函数来进一步操作。

In [5]:
import torch

X,y = torch.tensor(inputs.values),torch.tensor(outputs.values)
X,y

(tensor([[2., 1., 0.],
         [1., 1., 0.],
         [3., 0., 1.],
         [2., 0., 1.]], dtype=torch.float64),
 tensor([127500, 124500, 123200, 123200]))

### 2.2.4 练习

创建包含更多行和列的原始数据集。
1. 删除缺失值最多的列。
2. 将预处理后的数据集转换为张量格式。

In [6]:
print(data)

   NumRooms  Alley   Prices
0       NaN   Pave   127500
1       1.0   Pave   124500
2       3.0    NaN   123200
3       NaN    NaN   123200


In [7]:
data.count(axis='index')

NumRooms    2
 Alley      2
 Prices     4
dtype: int64

In [8]:
data.drop(data.count(axis='index').idxmin(),axis=1)

,Alley,Prices
0,Pave,127500
1,Pave,124500
2,NaN,123200
3,NaN,123200
